In [1]:
import os
import os.path
os.chdir('..')

In [6]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
from matplotlib import cm, colors
import pytorch_lightning as pl

from simclr_finetuner import SSLFineTuner
from simclr_module import SimCLR
from pointnet_module import SupervisedPointNet
from datasets.shapenet_parts.shapenet_parts import ShapeNetParts
from datasets.data_modules import PartSegmentationDataModule

from transforms import FineTuningTestDataTransform
from augmentations.augmentations import Rotation, Flip, Rescale, GaussianNoise

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
batch_size = 4
#transforms = [Rotation(0.5),Flip(1),Rescale(),GaussianNoise(0.7)]
transforms = [Rescale(0.5)]

backbone_ckpt = 'checkpoints/simclr_best_21071416584428.ckpt'
ssl_finetuner_ckpt = 'checkpoints/fine_tuning_best_21071514122946.ckpt'

backbone = SimCLR.load_from_checkpoint(backbone_ckpt)

for transform in transforms:
    current_transform = FineTuningTestDataTransform([transform])
    dm = PartSegmentationDataModule(batch_size,test_transforms=current_transform,fine_tuning=True)
    seg_class_map = dm.seg_class_map
    test_dataloader = dm.test_dataloader()
    model = SSLFineTuner.load_from_checkpoint(ssl_finetuner_ckpt, backbone=backbone, 
                                   seg_class_map=seg_class_map)    
    
    trainer = pl.Trainer(
        gpus=1,
        num_nodes=1,
        precision=16,
        max_epochs=1,
        distributed_backend=None,
        sync_batchnorm=False,
    )
    results = trainer.test(model = model, test_dataloaders = test_dataloader)
    print('Results for transform: {} \n{}'.format(str(transform),results))

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 100%|██████████| 719/719 [02:00<00:00,  5.96it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test mIoU of Airplane': 0.8126567006111145,
 'test mIoU of Bag': 0.7644801139831543,
 'test mIoU of Cap': 0.768993616104126,
 'test mIoU of Car': 0.7459132075309753,
 'test mIoU of Chair': 0.892654299736023,
 'test mIoU of Earphone': 0.666413426399231,
 'test mIoU of Guitar': 0.9003296494483948,
 'test mIoU of Knife': 0.8433716297149658,
 'test mIoU of Lamp': 0.788693368434906,
 'test mIoU of Laptop': 0.9481350183486938,
 'test mIoU of Motorbike': 0.5886881351470947,
 'test mIoU of Mug': 0.9115614891052246,
 'test mIoU of Pistol': 0.7887442708015442,
 'test mIoU of Rocket': 0.545494794845581,
 'test mIoU of Skateboard': 0.7295714020729065,
 'test mIoU of Table': 0.8141013979911804,
 'test_accuracy': 0.9304516315460205,
 'test_class_avg_accuracy': 0.8374872207641602,
 'test_class_avg_iou': 0.7818626761436462,
 'test_insta

In [12]:
# For supervised
batch_size = 4
#transforms = [Rotation(0.5),Flip(1),Rescale(0.5),GaussianNoise(0.7)]
transforms = [GaussianNoise(0.7)]

supervised_no_aug_ckpt = 'checkpoints/supervised_no_aug_best_21071320395666.ckpt'


for transform in transforms:
    current_transform = FineTuningTestDataTransform([transform])
    dm = PartSegmentationDataModule(batch_size,test_transforms=current_transform,fine_tuning=True)
    seg_class_map = dm.seg_class_map
    test_dataloader = dm.test_dataloader()    
    model = SupervisedPointNet.load_from_checkpoint(supervised_no_aug_ckpt,seg_class_map=seg_class_map)
    
    trainer = pl.Trainer(
        gpus=1,
        num_nodes=1,
        precision=16,
        max_epochs=1,
        distributed_backend=None,
        sync_batchnorm=False,
    )
    results = trainer.test(model = model, test_dataloaders = test_dataloader)
    print('Results for transform: {} \n{}'.format(str(transform),results))

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 100%|██████████| 719/719 [01:57<00:00,  6.13it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'eval mIoU of Airplane': 0.7789437174797058,
 'eval mIoU of Bag': 0.7955501079559326,
 'eval mIoU of Cap': 0.8566972017288208,
 'eval mIoU of Car': 0.7318137288093567,
 'eval mIoU of Chair': 0.8873791694641113,
 'eval mIoU of Earphone': 0.7098955512046814,
 'eval mIoU of Guitar': 0.8990153670310974,
 'eval mIoU of Knife': 0.8331015706062317,
 'eval mIoU of Lamp': 0.7836562395095825,
 'eval mIoU of Laptop': 0.9409699440002441,
 'eval mIoU of Motorbike': 0.6213475465774536,
 'eval mIoU of Mug': 0.904929518699646,
 'eval mIoU of Pistol': 0.7962162494659424,
 'eval mIoU of Rocket': 0.5957265496253967,
 'eval mIoU of Skateboard': 0.6944325566291809,
 'eval mIoU of Table': 0.8075169920921326,
 'test_accuracy': 0.926337480545044,
 'test_class_avg_accuracy': 0.8361691832542419,
 'test_class_avg_iou': 0.7898244857788086,
 'test_in